In [2]:
import os
os.environ["OTEL_SDK_DISABLED"] = "true"

In [3]:
import sys
print(sys.getrecursionlimit())

3000


In [4]:
import resource, sys
resource.setrlimit(resource.RLIMIT_STACK, (2**29,-1))
sys.setrecursionlimit(10**6)

In [ ]:
# crewai_preauth_provider.py
from crewai import Agent, Task, Crew
from crewai import LLM
from crewai.tools import tool

# ---------- TOOL SIMULATING INSURER SUBMISSION ----------
@tool
def submit_preauth_to_insurer(preauth_data: dict) -> str:
    """
    Simulates submission of a pre-auth form to the insurer's portal.
    """
    return f"✅ Preauth submitted successfully. Reference ID: {hash(str(preauth_data)) % 100000}"

# ---------- LLM SETUP ----------
llm = LLM(model="ollama/mistral:latest", base_url="http://localhost:11434")

# ---------- AGENTS ----------


front_registration_agent = Agent(
    role="Front Registration Desk Assistant",
    goal="Collect accurate patient demographics, insurance details, and clinical notes to initiate the preauthorization process.",
    backstory=(
        "You are an AI-assisted front desk coordinator in a hospital. "
        "Your job is to gather complete and accurate information from patients, "
        "including demographics, insurance card images, and the attending physician's clinical notes. "
        "You ensure that an Electronic Medical Record (EMR) is created or updated, and associate the patient with the correct physician and department. "
        "Once all required data is verified and structured, you pass it to the diagnosis_billing_agent for coding and cost estimation."
    ),
    verbose=True,
    llm=llm,
    input_keys=["patient_info", "insurance_card", "clinical_notes"]
)

diagnosis_billing_agent = Agent(
    role="AI-Powered Medical Plan Assistant",
    goal="Translate clinical documentation into standardized codes and generate accurate treatment cost estimates.",
    backstory=(
        "You are a clinical NLP expert agent embedded in a hospital's preauthorization pipeline. "
        "You process physician notes to extract key medical concepts such as diagnoses, procedures, and comorbidities. "
        "You then map these concepts to standard coding systems like ICD-10, CPT/HCPCS, and DRG where applicable. "
        "You also estimate procedure costs, length of stay, and post-treatment requirements based on internal hospital pricing databases and historical data. "
        "Your outputs include standardized codes, cost estimates, and a clinical justification summary, which are then forwarded to the Insurance Desk Agent."
    ),
    verbose=True,
    llm=llm,
    input_keys=["clinical_notes", "pricing_data", "coding_reference"]
)


insurance_desk_agent = Agent(
    role="AI-Powered Insurance Coordinator",
    goal="Validate patient insurance eligibility and coverage for the planned diagnosis and procedure.",
    backstory=(
        "You are an expert AI agent embedded in the hospital’s insurance desk workflow. "
        "Your job is to verify the patient’s insurance details using payer APIs, including policy status, plan coverage, and network affiliation. "
        "You cross-check the diagnosis and procedure codes from the Diagnosis Billing Agent against the patient’s plan to ensure coverage, "
        "flag any exclusions, and identify inconsistencies like inactive policies or mismatched patient names. "
        "Your validation ensures only complete and accurate requests move forward for preauthorization submission."
    ),
    verbose=True,
    llm=llm,
    input_keys=["insurance_card", "policy_eligibility", "coding_reference"]
)

preauth_form_generator_agent = Agent(
    role="AI-Powered Preauthorization Compiler and Submitter",
    goal="Assemble, format, and submit a compliant preauthorization request to the insurer.",
    backstory=(
        "You are a final-stage automation agent in the hospital’s preauthorization workflow. "
        "You gather structured and validated data from upstream agents—including patient demographics, insurance details, diagnosis codes, procedure codes, cost estimates, and clinical justification. "
        "You format this data according to payer-specific standards such as EDI 278, FHIR bundles, PDFs, or web form templates. "
        "After submission via secure portals or APIs, you log the reference ID, timestamp, and confirmation receipt. "
        "You notify relevant stakeholders and ensure full traceability of the preauth submission."
    ),
    verbose=True,
    llm=llm,
    input_keys=[
        "patient_info", "insurance_card", "pricing_data",
        "coding_reference", "insurer_submission"
    ]
)


# ---------- TASKS ----------

collect_patient_info = Task(
    description="Collect patient demographics such as first name, last name, dob, gender, and contact details from the input. \
       Input: {patient_info} ",
    # Note: This task simulates the collection of patient information
    # In a real-world scenario, it would involve form filling or database entry.
    # TODO: Connect to DB and store in database. 
    expected_output="Structured patient demographic data in EMR format.",
    agent=front_registration_agent
)

scan_insurance_card = Task(
    description="Scan and digitize both sides of the patient's insurance card. Extract policy number, insurer name, and group ID from the input. \
       Input: {insurance_card} ",
    # Note: This task simulates the scanning and extraction of insurance card details
    # Add OCR processing or image analysis.
    expected_output="JSON with insurance card details attached to patient's EMR.",
    agent=front_registration_agent
)

gather_clinical_notes = Task(
    description="Receive and store initial physician documentation, including presenting symptoms, working diagnosis, and treatment recommendation. \
        Input Clinical notes: {clinical_notes} ",
        
    expected_output="Structured clinical note summary linked to the patient’s record.",
    agent=front_registration_agent
)

extract_clinical_concepts = Task(
    description="Extract key clinical concepts such as diagnoses, procedures, and comorbidities. Use NLP to extract diagnosis, procedure, and comorbidities from physician notes. \
        Input: {clinical_notes} ",
    expected_output="A list of clinical concepts extracted.",
    agent=diagnosis_billing_agent
)

generate_standard_codes = Task(
    description="Map clinical concepts to ICD-10, CPT/HCPCS, and DRG codes using the standard coding rules. \
        Coding reference: {coding_reference} ",
        # Note: This task simulates the coding process
        # In a real-world scenario, it would involve querying coding databases or APIs.
        # TODO: Implement coding logic
    expected_output="A Mapping of codes for diagnosis and procedures.",
    agent=diagnosis_billing_agent
)

estimate_treatment_costs = Task(
    description="Estimate total treatment cost based on procedure, length of stay, and consumables using hospital pricing data. \
        Reference treatment cost data: {pricing_data} ",
        # Note: This task simulates cost estimation
        # In a real-world scenario, it would involve querying hospital pricing databasess
    expected_output="Itemized cost estimation in structured format.",
    agent=diagnosis_billing_agent
)

summarize_clinical_justification = Task(
    description="Generate a guideline-compliant clinical justification summary for the planned treatment. \
        Clinical notes: {clinical_notes} ",
    expected_output="Clear justification aligned with MCG or InterQual criteria.",
    agent=diagnosis_billing_agent
)

# check_policy_eligibility = Task(
#     description="Validate insurance eligibility using X12 270/271 or FHIR API. Confirm active policy status, network match, and patient name consistency. \
#         Input: {insurance_card} {policy_eligibility} ",
#     expected_output="Eligibility verification report with API response.",
#     agent=insurance_desk_agent
# )

# validate_coverage = Task(
#     description="Cross-check diagnosis and procedure codes against patient's insurance plan for inclusion, exclusions, or limits. \
#             Input: {policy_eligibility} {coding_reference} ",
#     expected_output="Coverage validation summary with rule references.",
#     agent=insurance_desk_agent
# )

# detect_inconsistencies = Task(
#     description="Identify issues such as expired policy, non-covered services, or mismatched diagnosis codes. Alert stakeholders. \
#             Input: {policy_eligibility} {insurance_card} {coding_reference} ",
#     expected_output="List of flagged inconsistencies with severity and resolution suggestions.",
#     agent=insurance_desk_agent
# )

merge_patient_and_treatment_data = Task(
    description="Merge data from prior agents including patient info, insurance, diagnosis/procedure codes, cost estimate, and justification. \
        ",
    expected_output="Preauthorization data packet ready for formatting.",
    agent=preauth_form_generator_agent
)

format_preauth_form = Task(
    description="Format the compiled data into insurer-compliant templates (EDI 278, FHIR bundle, PDF, or web form). \
        ",
    expected_output="Well-structured and compliant preauth request document.",
    input_keys=["insurer_submission"],
    agent=preauth_form_generator_agent
)

# submit_to_insurer = Task(
#     description="Submit the formatted preauth form to the insurer’s portal or API and handle any real-time response or acknowledgment. \
#         Insuer submission details: {insurer_submission} ",
#     # Note: This task simulates the submission process
#     # In a real-world scenario, it would involve API calls or EDI transactions.
#     expected_output="Submission confirmation with unique reference ID.",
#     agent=preauth_form_generator_agent
# )

# log_submission_reference = Task(
#     description="Log the submission details including timestamp, reference number, and notify relevant teams (front desk, physician). \
#             Insurer submission details: {insurer_submission} ",
#     # Note: This task simulates the logging process
#     # In a real-world scenario, it would involve database or audit log updates.
#     expected_output="Audit log record and team notifications sent.",
#     agent=preauth_form_generator_agent
# )


# ---------- CREW SETUP ----------

provider_preauth_crew = Crew(
    agents=[
        front_registration_agent,
        diagnosis_billing_agent,
        insurance_desk_agent,
        preauth_form_generator_agent
    ],
    tasks=[
        collect_patient_info,
        scan_insurance_card,
        gather_clinical_notes,

        extract_clinical_concepts,
        generate_standard_codes,
        estimate_treatment_costs,
        summarize_clinical_justification,

        # check_policy_eligibility,
        # validate_coverage,
        # detect_inconsistencies,

        merge_patient_and_treatment_data,
        format_preauth_form,
        submit_to_insurer,
        log_submission_reference
    ],
    model="ollama/mistral:latest",
    verbose=False,
    process_name="Hospital PreAuthorization Provider-Side Automation"
)

# ---------- RUN WORKFLOW ----------
if __name__ == "__main__":
    input = {
    "patient_info": {
        "first_name": "Rajesh",
        "last_name": "Kumar",
        "dob": "1980-05-12",
        "gender": "Male",
        "contact": {
            "phone": "+91-9876543210",
            "email": "rajesh.kumar@example.com"
        },
        "address": "Hyderabad, Telangana, India",
        "ehr_id": "EHR-998877",
        "physician": {
            "name": "Dr. Anjali Mehta",
            "department": "Orthopedics",
            "npi": "1234567890"
        }
    },
    "insurance_card": {
        "insurer_name": "Star Health Insurance",
        "policy_number": "STAR-HI-20394857",
        "group_number": "GRP-112233",
        "valid_from": "2024-01-01",
        "valid_to": "2025-12-31",
        "network_affiliation": "In-Network"
    },
    "clinical_notes": {
        "symptoms": "Severe pain and stiffness in left knee",
        "preliminary_diagnosis": "Left Knee Osteoarthritis",
        "recommended_procedure": "Arthroscopic Knee Surgery",
        "risk_factors": ["Hypertension", "Obesity"],
        "notes_text": """
            The patient presents with chronic left knee pain for over 6 months, worsening with movement. 
            X-ray confirms osteoarthritis. Recommended arthroscopic intervention with pre/post rehab. 
            No known drug allergies. Pre-op evaluation needed.
        """
    },
    "pricing_data": {
        "procedure_name": "Arthroscopic Knee Surgery",
        "procedure_code": "29881",
        "procedure_cost": 48000,
        "estimated_length_of_stay": "2 days",
        "pre_post_care_cost": 7500,
        "consumables": 3200,
        "total_estimate": 58700
    },
    "coding_reference": {
        "icd_codes": {
            "M17.12": {
            "description": "Unilateral primary osteoarthritis, left knee",
            "category": "Disorder",
            "related_symptoms": [
                "knee pain",
                "joint stiffness",
                "reduced mobility"
            ],
            "associated_procedures": [
                "Arthroscopy",
                "Knee replacement"
            ]
            }
        },
        "cpt_codes": {
            "29881": {
            "description": "Arthroscopy, knee, surgical; with meniscectomy (medial OR lateral including any meniscal shaving)",
            "category": "Procedure",
            "used_for": [
                "Meniscal tear",
                "Osteoarthritis",
                "Knee pain"
            ],
            "typical_duration": "1 hour"
            }
        },
        "drg_code": {
            "470": {
            "description": "Major joint replacement or reattachment of lower extremity",
            "category": "Procedure Grouping",
            "covered_procedures": [
                "Total Knee Replacement",
                "Hip Replacement"
            ],
            "hospital_stay_avg": "3-4 days"
            }
        }
    },
    "policy_eligibility": {
        "active": True,
        "policyholder_name_matches": True,
        "coverage_status": "Approved for orthopedic procedures",
        "pre_existing_clauses": False,
        "deductible_remaining": 5000,
        "copay": 1200
    },
    "insurer_submission": {
        "method": "FHIR",
        "endpoint_url": "https://localhost:2624/priorauth",
        "submission_format": "FHIR PriorAuth Bundle",
        "portal_backup": True
    }
}

    
    result = provider_preauth_crew.kickoff(inputs=input)
    print("\n=== Final Output ===")
    print(result)


# Agent: Front Registration Desk Assistant
## Task: Collect patient demographics such as first name, last name, dob, gender, and contact details from the input.        Input: {'first_name': 'Rajesh', 'last_name': 'Kumar', 'dob': '1980-05-12', 'gender': 'Male', 'contact': {'phone': '+91-9876543210', 'email': 'rajesh.kumar@example.com'}, 'address': 'Hyderabad, Telangana, India', 'ehr_id': 'EHR-998877', 'physician': {'name': 'Dr. Anjali Mehta', 'department': 'Orthopedics', 'npi': '1234567890'}} 


# Agent: Front Registration Desk Assistant
## Final Answer: 
The patient demographic data collected from the input has been successfully structured in the Electronic Medical Record (EMR) format as follows:

Patient Name: Rajesh Kumar
Date of Birth: 1980-05-12
Gender: Male
Contact Details:
Phone: +91-9876543210
Email: rajesh.kumar@example.com
Address: Hyderabad, Telangana, India
EHR ID: EHR-998877
Physician Information:
Name: Dr. Anjali Mehta
Department: Orthopedics
NPI: 1234567890

The patient d

KeyboardInterrupt: 